In [1]:
# https://blog.naver.com/ostin1038/222135543376

In [2]:
import pandas as pd
import numpy as np

In [4]:
D = pd.read_csv("Human_Resources_Employee_Attrition.csv")

In [5]:
D.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
satisfaction_level       14999 non-null float64
last_evaluation          14999 non-null float64
number_of_projects       14999 non-null int64
average_monthly_hours    14999 non-null int64
years_at_company         14999 non-null int64
work_accident            14999 non-null int64
left                     14999 non-null int64
promotion_last_5years    14999 non-null int64
department               14999 non-null object
salary                   14999 non-null object
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


In [6]:
D.describe()

,satisfaction_level,last_evaluation,number_of_projects,average_monthly_hours,years_at_company,work_accident,left,promotion_last_5years
count,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000
mean,0.612834,0.716102,3.803054,201.050337,3.498233,0.144610,0.238083,0.021268
std,0.248631,0.171169,1.232592,49.943099,1.460136,0.351719,0.425924,0.144281
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


In [7]:
D.head()

,satisfaction_level,last_evaluation,number_of_projects,average_monthly_hours,years_at_company,work_accident,left,promotion_last_5years,department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [8]:
# Q1 부서별 퇴직률 비율(left=1 / 부서 전체 인원)이 가장 높은 부서를 확인하고자 한다. 
#  퇴직률이 가장 높은 부서를 확인하고 퇴직률을 서술하시오(소수 셋째자리 버림, 둘째자리까지)

In [10]:
D1 = D.groupby("department").left.agg(['count',  'sum'])
D1

,count,sum
department,,
IT,1227,273
RandD,787,121
accounting,767,204
hr,739,215
management,630,91
marketing,858,203
product_mng,902,198
sales,4140,1014
support,2229,555


In [13]:
D1['rate']= D1['sum'] / D1['count']
D1.sort_values(by='rate',ascending=False)

,count,sum,rate
department,,,
hr,739,215,0.290934
accounting,767,204,0.265971
technical,2720,697,0.256250
support,2229,555,0.248991
sales,4140,1014,0.244928
marketing,858,203,0.236597
IT,1227,273,0.222494
product_mng,902,198,0.219512
RandD,787,121,0.153748


In [14]:
# Q2
# 1번을 통해 확인한 퇴직률이 가장 높은 부서와 낮은 부서의 만족도가 실제로 통계적으로 차이가 있는지 따져보고자 한다. 
# 만족도, 최근평가, 월평균 근무시간 3개 수치형변수에 대해 모두 독립표본 T검정을 수행하고 
# 3개 변수 중 통계적으로 차이가 있는 변수는 모두 몇개인지 서술하시오(Confidence Level 0.95) 


In [18]:
# 퇴직율 고:hr 저:management
D[D.department == 'hr'].satisfaction_level.values.shape

(739,)

In [19]:
D[D.department == 'management'].satisfaction_level.values.shape

(630,)

In [20]:
from scipy.stats import ttest_ind

In [21]:
ttest_ind(D[D.department == 'hr'].satisfaction_level, D[D.department == 'management'].satisfaction_level)

Ttest_indResult(statistic=-1.7402494455498025, pvalue=0.08204025790123437)

In [22]:
# Q3
# Q3. Naive Bayes 모델을 활용하여 퇴직자 예측 알고리즘을 만들고자 한다. 
# 모델을 만들기 위해 아래 3가지의 전처리를 수행한다. 

# 먼저 퇴직율 비율이 높은 부서 5개에 속하는 여부에 따라 수치형 변수를 만든다. (속하면=1, 아니면=0) 
# 다음 연봉 변수를 더미변수로 전환한다 (범주 수 -1 적용, 연봉 medium에 해당하는 변수 제거) 
# 마지막으로 프로젝트 수, 월 근무시간, 근무연수 3개 수치형 변수를 정규화한다.(MinMaxScaler방법 활용)
# 트레인셋, 테스트셋을 7:3 분할로 분할한 후 (seed:13) Naive Bayes 모델을 만든다. 

# 독립변수: 만족도, 최근 평가, 사고 유무, 최근 5년 내 승진여부, 퇴직율 비율 높은 부서 속하는 여부, 
#           연봉 더미변수, 정규화한 3가지 변수(프로젝트 수, 월 근무시간, 근무연수) 

# 종속변수: 퇴직여부 

In [31]:
high_5 = D1.sort_values(by='rate',ascending=False).iloc[0:5].index.values
high_5

array(['hr', 'accounting', 'technical', 'support', 'sales'], dtype=object)

In [30]:
D3 = D.copy()

In [38]:
np.where(D3.department.isin(high_5), "1", "0")

array(['1', '1', '1', ..., '1', '1', '1'], dtype='<U1')

In [42]:
D3['high5_department'] = np.where(D3.department.isin(high_5), 1, 0)

In [51]:
D3[D3.high5_department==1].groupby('department').agg('mean')

,satisfaction_level,last_evaluation,number_of_projects,average_monthly_hours,years_at_company,work_accident,left,promotion_last_5years,high5_department
department,,,,,,,,,
accounting,0.582151,0.717718,3.825293,201.162973,3.522816,0.125163,0.265971,0.018253,1
hr,0.598809,0.708850,3.654939,198.684709,3.355886,0.120433,0.290934,0.020298,1
sales,0.614447,0.709717,3.776329,200.911353,3.534058,0.141787,0.244928,0.024155,1
support,0.618300,0.723109,3.803948,200.758188,3.393001,0.154778,0.248991,0.008973,1
technical,0.607897,0.721099,3.877941,202.497426,3.411397,0.140074,0.256250,0.010294,1


In [54]:
salary_level = pd.get_dummies(D3.salary).drop('medium', axis=1)

In [55]:
from sklearn.preprocessing import MinMaxScaler

In [56]:
scaler = MinMaxScaler()

In [60]:
normalized_inds = scaler.fit_transform(D3[['number_of_projects','average_monthly_hours','years_at_company' ]])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [69]:
normalized_df = pd.DataFrame(data= normalized_inds, columns=['number_of_projects','average_monthly_hours','years_at_company'])

In [71]:
# 독립변수: 만족도, 최근 평가, 사고 유무, 최근 5년 내 승진여부, 퇴직율 비율 높은 부서 속하는 여부, 
#           연봉 더미변수, 정규화한 3가지 변수(프로젝트 수, 월 근무시간, 근무연수) 
D32 = pd.concat([
    D3[['satisfaction_level','last_evaluation','work_accident','promotion_last_5years','high5_department']],
    salary_level,
    normalized_df,
], axis=1)

In [72]:
D32.head()

,satisfaction_level,last_evaluation,work_accident,promotion_last_5years,high5_department,high,low,number_of_projects,average_monthly_hours,years_at_company
0,0.38,0.53,0,0,1,0,1,0.0,0.285047,0.125
1,0.80,0.86,0,0,1,0,0,0.6,0.775701,0.500
2,0.11,0.88,0,0,1,0,0,1.0,0.822430,0.250
3,0.72,0.87,0,0,1,0,1,0.6,0.593458,0.375
4,0.37,0.52,0,0,1,0,1,0.0,0.294393,0.125


In [73]:
D32.shape

(14999, 10)

In [75]:
D3['left'].head()

0    1
1    1
2    1
3    1
4    1
Name: left, dtype: int64

In [76]:
from sklearn.model_selection import train_test_split

In [82]:
X_train, X_test, y_train, y_test = train_test_split(D32, D3['left'], test_size=0.3, random_state=13)

In [83]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((10499, 10), (4500, 10), (10499,), (4500,))

In [84]:
from sklearn.naive_bayes import MultinomialNB

In [85]:
model = MultinomialNB()

In [86]:
model.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [87]:
pred = model.predict(X_test)

In [88]:
pd.crosstab(pred, y_test) # 분류

left,0,1
row_0,,
0,3434,1066


In [93]:
proba = model.predict_proba(X_test)
proba

array([[0.73160555, 0.26839445],
       [0.75772932, 0.24227068],
       [0.75860339, 0.24139661],
       ...,
       [0.72098153, 0.27901847],
       [0.74763618, 0.25236382],
       [0.61972695, 0.38027305]])

In [91]:
model.classes_

array([0, 1], dtype=int64)

In [94]:
pd.DataFrame(data=proba).describe()

,0,1
count,4500.000000,4500.000000
mean,0.767211,0.232789
std,0.092890,0.092890
min,0.596101,0.004054
25%,0.697702,0.207952
50%,0.748183,0.251817
75%,0.792048,0.302298
max,0.995946,0.403899


In [97]:
new_pred = np.where(proba[:,1]>0.3, 1, 0)

In [100]:
pd.crosstab(new_pred, y_test)

left,0,1
row_0,,
0,2841,472
1,593,594


In [101]:
from sklearn.metrics import f1_score

In [102]:
f1_score(y_test, new_pred)

0.5272969374167775